In [ ]:
#The import cell: You'll only need to run this cell once, but you'll need to run it first
# To run, hold shift and press Enter.
import hoomd
import hoomd.md
from hoomd import deprecated
import mbuild, mdtraj, nglview, numpy
import matplotlib.pyplot as plt
%matplotlib inline
Number_of_particles = 200 
def quickrun(kT,AA,AB,BB,name):
    Temperature = kT
    hoomd.context.initialize("")
    system=hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=2,a1=[2.2,0,0],a2=[0,1.1,0],a3=[0,0,1],dimensions=2,type_name=["A","B"],position=[[0,0,0],[1.1,0,0]]),n=10)
    deprecated.dump.xml(filename="init.hoomdxml",group=hoomd.group.all(),vis=True)
    nl = hoomd.md.nlist.tree()
    lj = hoomd.md.pair.lj(r_cut=2.5,nlist=nl)
    lj.pair_coeff.set("A","A",epsilon=AA,sigma=1)
    lj.pair_coeff.set("A","B",epsilon=AB,sigma=1)
    lj.pair_coeff.set("B","B",epsilon=BB,sigma=1)
    hoomd.md.integrate.mode_standard(dt=0.007)
    all=hoomd.group.all()
    hoomd.md.integrate.langevin(group=all,kT=Temperature,seed=14)
    hoomd.dump.dcd(filename=name+".dcd",period=50,group=hoomd.group.all(),overwrite=True)
    hoomd.analyze.log(filename=name+"-log",quantities=['potential_energy','temperature','pressure','kinetic_energy'],period=50,overwrite=True)
    hoomd.run(5e4)
    deprecated.dump.xml(filename=name+".hoomdxml",group=hoomd.group.all(),vis=True)
    
    

In [ ]:
#Ideal Gas
kT=0.9
name="IDEAL_GAS"
quickrun(kT,0,0,0,name) #Temperature, AA, AB, BB, and run name
t = mdtraj.load(name+".dcd",top=name+".hoomdxml")
view = nglview.show_mdtraj(t)
view.clear_representations()
view.add_representation("point",selection=[i*2 for i in range(Number_of_particles)],color='blue',pointSize=2.5,useTexture=True)
view.add_representation("point",selection=[i*2+1 for i in range(Number_of_particles)],color='orange',pointSize=2.5,useTexture=True)
print(name)
data=numpy.loadtxt(name+"-log",skiprows=1)
plt.plot(data[:,0],data[:,1]+data[:,4]) #Total energy vs time step
plt.ylabel("Total Energy")
plt.ylabel("time step")
print("Average TE: {}".format(numpy.mean(data[:,1]+data[:,4])))
view

In [ ]:
#Identical Lennard-Jones Particles
#Ideal Gas
kT=0.9
name="LENNARD_JONES"
quickrun(kT,1,1,1,name) #Temperature, AA, AB, BB, and run name
t = mdtraj.load(name+".dcd",top=name+".hoomdxml")
view = nglview.show_mdtraj(t)
view.clear_representations()
view.add_representation("point",selection=[i*2 for i in range(Number_of_particles)],color='blue',pointSize=2.5,useTexture=True)
view.add_representation("point",selection=[i*2+1 for i in range(Number_of_particles)],color='orange',pointSize=2.5,useTexture=True)
print(name)
data=numpy.loadtxt(name+"-log",skiprows=1)
plt.plot(data[:,0],data[:,1]+data[:,4]) #Total energy vs time step
plt.ylabel("Total Energy")
plt.ylabel("time step")
print("Average TE: {}".format(numpy.mean(data[:,1]+data[:,4])))
view

In [ ]:
#Demixing Lennard-Jones (A does not Like B)
#Ideal Gas
kT=0.9
name="Demix"
quickrun(kT,1,0.1,1,name) #Temperature, AA, AB, BB, and run name
t = mdtraj.load(name+".dcd",top=name+".hoomdxml")
view = nglview.show_mdtraj(t)
view.clear_representations()
view.add_representation("point",selection=[i*2 for i in range(Number_of_particles)],color='blue',pointSize=2.5,useTexture=True)
view.add_representation("point",selection=[i*2+1 for i in range(Number_of_particles)],color='orange',pointSize=2.5,useTexture=True)
print(name)
data=numpy.loadtxt(name+"-log",skiprows=1)
plt.plot(data[:,0],data[:,1]+data[:,4]) #Total energy vs time step
plt.ylabel("Total Energy")
plt.ylabel("time step")
print("Average TE: {}".format(numpy.mean(data[:,1]+data[:,4])))
view

In [ ]:
#calculate heat capacity
#Identical Lennard-Jones Particles
#Ideal Gas
Temps=numpy.arange(0.5,2.0,0.25)
Energies = []
Std = []
name="IDEAL_GAS"
for T in Temps:
    quickrun(T,0,0,0,name) #Temperature, AA, AB, BB, and run name
    data=numpy.loadtxt(name+"-log",skiprows=1)
    Energies.append(numpy.average(data[:,1]+data[:,4]))
    Std.append(numpy.std(data[:,1]+data[:,4]))

In [ ]:
plt.plot(Temps,Energies) 
A = numpy.vstack([Temps, numpy.ones(len(Temps))]).T #some numpy magic needed for...
m,b = numpy.linalg.lstsq(A,Energies,rcond=None)[0]  #doing the linear regression (y=mx+b)
plt.plot(Temps,Energies,'o')
plt.plot(Temps,m*Temps+b,'r')
plt.xlabel("kT")
plt.ylabel("TE")
print("Slope, m={}".format(m))